In [1]:
import os
os.chdir(r'G:\Kashif_OpenEnded') 
from stlf_torch_kit import  DataLoadeing
import torch
import pandas as pd
import numpy as np
import pickle, time
from stlf_torch_kit import univariate_multi_step
from stlf_torch_kit import SaveBestModel, PlotLossCurves, LoadModel, train, TestModel, BatchGenerator, results
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

# Data Loading

In [2]:
import os
path_dataset ='G:\Kashif_OpenEnded\Dataset'
path_tr = os.path.join(path_dataset, 'CityLearn_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'CityLearn_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'Citylearn_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((6132, 29), (1752, 29), (876, 29))

In [3]:
time_steps = 24
target_len = 1 #how much steps do you wana forecast #Edit
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=target_len)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=target_len)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=target_len)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.10862445831298828 sec


In [21]:
test_X.shape

(3989, 36, 21)

#### Mostly Cited

In [12]:
class MOSTLY_CITED(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.lstm = nn.LSTM(29, 60, 3, batch_first=True, dropout=0.2).to(self.device)  # Added dropout
        self.layer_norm = nn.LayerNorm(60).to(self.device)  # Added LayerNorm
        self.fc = nn.Linear(60, 1).to(self.device)

        for name, param in self.lstm.named_parameters():
            if 'weight_ih' in name or 'weight_hh' in name:
                torch.nn.init.xavier_normal_(param.data)
            elif 'bias' in name:
                nn.init.zeros_(param)

    def forward(self, x):
        h0 = torch.zeros(3, x.size(0), 60).to(self.device)
        c0 = torch.zeros(3, x.size(0), 60).to(self.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.layer_norm(out[:, -1, :])  # Apply LayerNorm
        out = self.fc(out)  # Removed sigmoid activation
        return out


In [4]:
class MOSTLY_CITED1(nn.Module):
    def __init__(self, input_size=29, hidden_size=20, num_layers=2):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        
        # Move all parameters to device
        self.to(self.device)

    def forward(self, x):
        # Move input to same device as model
        x = x.to(self.device)
        out, _ = self.lstm(x)  # out shape: (batch, seq_len, hidden_size)
        out = self.fc(out[:, -1, :])  # Take last timestep
        return out

# instances

In [5]:
model = MOSTLY_CITED1()#Edit
criterion = nn.MSELoss() #Edit, don't change

save_best_model = SaveBestModel()
Plot_Loss=PlotLossCurves()
load_model=LoadModel()

# Learning Rate & Optimizer

In [6]:
#lr=0.001 # Edit
#optimizer = torch.optim.Adam(model.parameters(), lr=lr) #Edit
lr = 0.001  # Initial learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)  # Added


# Check Device

In [7]:
def get_model_device(model):
    return next(model.parameters()).device
device = get_model_device(model)
print("Model is on device:", device)

Model is on device: cpu


# Training

#### Path & other Stuffs

In [8]:
start_epoch = 1
num_epochs = 20 #Edit
best_model_path=r'G:\Kashif_OpenEnded\chk'+str('\\') #Edit
fig_path=r'G:\Kashif_OpenEnded\chk' #Edit
train_data_loader, validation_data_loader, test_data_loader = DataLoadeing(train_X ,
                                                                           train_y, 
                                                                           validation_X, 
                                                                           validation_y, 
                                                                           test_X, 
                                                                           test_y, 
                                                                           batch_size=32) #Batch_Size Edit

#### Instances

In [9]:
criterion = nn.MSELoss() # Edit, for Now Don't  Change

criterion_mae = nn.L1Loss()

save_best_model = SaveBestModel()
Plot_Loss=PlotLossCurves()
load_model=LoadModel()

#### Training Loop

In [10]:
start = time.time()
train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader)
print('Time Consumed', time.time()-start, "sec")

Epoch [1/20], Step [191/191], Training Loss: 0.0540
Epoch [1/20], Step [54/54], Val Loss: 0.0402

Saving best model for epoch: 1
 at G:\Kashif_OpenEnded\chk\1best_model.pth
Epoch [2/20], Step [191/191], Training Loss: 0.0401
Epoch [2/20], Step [54/54], Val Loss: 0.0403
Epoch [3/20], Step [191/191], Training Loss: 0.0383
Epoch [3/20], Step [54/54], Val Loss: 0.0405
Epoch [4/20], Step [191/191], Training Loss: 0.0370
Epoch [4/20], Step [54/54], Val Loss: 0.0409
Epoch [5/20], Step [191/191], Training Loss: 0.0360
Epoch [5/20], Step [54/54], Val Loss: 0.0414
Epoch [6/20], Step [191/191], Training Loss: 0.0350
Epoch [6/20], Step [54/54], Val Loss: 0.0416
Epoch [7/20], Step [191/191], Training Loss: 0.0340
Epoch [7/20], Step [54/54], Val Loss: 0.0417
Epoch [8/20], Step [191/191], Training Loss: 0.0327
Epoch [8/20], Step [54/54], Val Loss: 0.0417
Epoch [9/20], Step [191/191], Training Loss: 0.0314
Epoch [9/20], Step [54/54], Val Loss: 0.0414
Epoch [10/20], Step [191/191], Training Loss: 0.029

In [36]:
1240/60

20.666666666666668

#### Results

In [11]:
load_model_path=r'G:\Kashif_OpenEnded\chk\20best_model.pth' # Edit
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

# MAPE, MAE, RMSE

New lr = 0.001
Time Consumed 0.1435375213623047 sec
Mean Absolute Error (MAE): 0.38
Median Absolute Error (MedAE): 0.29
Mean Squared Error (MSE): 0.25
Root Mean Squared Error (RMSE): 0.5
Mean Absolute Percentage Error (MAPE): 61.16 %
Median Absolute Percentage Error (MDAPE): 41.01 %


y_test_unscaled.shape=  (851, 1)
y_pred.shape=  (851, 1)
